# Introduction: GNN for molecules

This example will showcase how to define and evaluate a GNN-like model in PyNeuraLogic on a sample molecule classification task.

Install PyNeuraLogic from PyPI

In [ ]:
! pip install neuralogic

## A Relational Dataset

We are going to use one of the predefined datasets called [Mutagenesis](https://www.doc.ic.ac.uk/~shm/mutagenesis.html). This popular dataset contains information about molecules that we are going to classify for mutagenicity on Salmonella typhimurium.

Predefined datasets are located in `neuralogic.utils.data`, and by calling `Mutagenesis`, we retrieve a tuple containing the input data (dataset) as well as a default template (rules). For this example, we are going to ignore the template as we will practice defining our own.

In [ ]:
from neuralogic.utils.data import Mutagenesis

_, dataset = Mutagenesis()

Predefined examples are loaded from a file. We can check the first sample to get an idea how the data look like.

In [ ]:
with open(dataset.examples_file) as fp:
    print(fp.readline())

As can be seen, the molecules are essentially encoded as heterogeneous graphs with edges called `bonds`, where the first two terms are the chemical atom ids, and the third term is the bond id. The atoms' ids are then associated with specific atom types, such as `h` for hydrogen. Bond ids are then associated with bond types, such as `b_1` for a single bond, etc.

\* Note that this textual storage format is just a bit more compact version of the way we would encode the data directly in the PyNeuraLogic language (Python), where we would instead write:

```
Relation.bond(Term.d59_23, Term.d59_5, 0), Relation.h(Term.d59_23), ...
```

Next we take a look at the associated *queries* which are the target labels associated with the examples in supervised learning scenarios.

In [ ]:
with open(dataset.queries_file) as fp:
    print(fp.readline())

we can see that each example molecule is associated with just a simple binary label called 'predict' here.

<sup>
* Note that in general you can associate each example with multiple different queries (e.g. joint classification), and that these can also be structured.
<sup>

## A Relational Template

The specification of the learning template is now up to the user. This is where you can get creative, but any valid template has to somehow connect the input example representations with the output queries (i.e. the 'predict' label).
In this example, we are going to first turn the atom and bond types into embeddings, and then propagate these jointly through the molecular graphs in a GNN-like fashion.

So let's start by embedding each chemical atom type into an `atom_embed` relation with a unique learnable parameter vector of shape `[3]` (a 3-dimensional embedding).
Doing this for each type can be simplified with list comprehension.

In [ ]:
from neuralogic.core import Template, R, V

template = Template()

template.add_rules([
    (R.atom_embed(V.A)[3,] <= R.get(atom)(V.A)) for atom in ["c", "o", "br", "i", "f", "h", "n", "cl"]
])

print(template)

In the same way, we can encode the embeddings for the bonds.

In [ ]:
template.add_rules([
    (R.bond_embed(V.B)[3,] <= R.get(bond)(V.B)) for bond in ["b_1", "b_2", "b_3", "b_4", "b_5", "b_7"]
])

Next we add the GNN-like graph propagation rule itself.
Particularly, we will want to project representations of all the nodes (`atom_embed`) through a learnable matrix of shape `[3, 3]`.
We will do this separately for the "central" node `X` and its neighbors `Y` (nodes connected by the `bond` relation) to accommodate for their different roles here.
Since we also have different bond types here, we will combine these atom projections together with the bond embedding.
For each central node `X`, we will then aggregate these representations from all the corresponding neighbors `Y`, to serve as the new representation for the central node `X`.
Let's call this resulting representation `layer_1`.

In [ ]:
template +=  R.layer_1(V.X) <= (R.atom_embed(V.X)[3, 3], R.atom_embed(V.Y)[3, 3], R.bond(V.X, V.Y, V.B), R.bond_embed(V.B))

In the same way, we can now add some more "layers" that will be 'connected' by utilizing the previously defined representations.

In [ ]:
template +=  R.layer_2(V.X) <= (R.layer_1(V.X)[3, 3], R.layer_1(V.Y)[3, 3], R.bond(V.X, V.Y, V.B), R.bond_embed(V.B))
template +=  R.layer_3(V.X) <= (R.layer_2(V.X)[3, 3], R.layer_2(V.Y)[3, 3], R.bond(V.X, V.Y, V.B), R.bond_embed(V.B))

Finally, to obatin a single label for the whole graph, we then aggregate all the nodes' (atoms') representations from the last layer (`layer_3`), and project them through another learnable vector of shape `[1, 3]` into a scalar value.
We will call the resulting representation `predict` to correspond to the learning target queries given in the dataset.

In [ ]:
template += R.predict[1, 3] <= R.layer_3(V.X)

print(template)

We can also take a look at the template visually.

In [ ]:
template.draw()

## Training

When we have our dataset and template ready, it's time to build ("ground") the template over the dataset and start training.
We can do the training manually and write our own custom training loop, but we can also use predefined helpers - *evaluators*,
that handle model and dataset building, training, and more. Evaluators can be customized via `Settings`.

<sup>Note that building the dataset (=grounding the logic rules and translating into neural networks) may take a while, depending on the complexity of your template.
But this is only done once before the training itself, which takes up most of the time anyway.
<sup>

In [ ]:
from neuralogic.core import Settings
from neuralogic.nn.loss import MSE, CrossEntropy
from neuralogic.nn import get_evaluator
from neuralogic.optim import Adam

settings = Settings(optimizer=Adam(lr=0.001), epochs=100, error_function=MSE())
evaluator = get_evaluator(template, settings)

built_dataset = evaluator.build_dataset(dataset)

Finally, we iterate through the iterator encapsulated in the `train` method of the evaluator, which yields a total loss of the epoch and the number of samples of the current epoch.
We then get access to the results from the training loop that we can further visualize, inspect, log, etc.

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt

average_losses = []

for current_total_loss, number_of_samples in evaluator.train(built_dataset):
    clear_output(wait=True)
    plt.ylabel("Loss")
    plt.xlabel("Epoch")

    plt.xlim(0, settings.epochs)

    average_losses.append(current_total_loss / number_of_samples)
    
    plt.plot(average_losses, label="Average loss")

    plt.legend()
    plt.pause(0.001)
    plt.show()

We can then check the trained model predictions (for the same sample set here) by utilizing the `test` method.

In [ ]:
for sample, y_hat in zip(built_dataset.samples, evaluator.test(built_dataset)):
    print(f"Target: {sample.java_sample.target}, Predicted: {round(y_hat)} ({y_hat})")